In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

C:\Users\User\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [3]:
df = pd.read_csv('FastText.csv')
df.head()

,0,0.1,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,0,0.000337,-0.000189,-0.000163,0.000118,-0.000304,-0.000059,0.000008,-0.000269,0.000145,...,-0.000250,-0.000002,0.000001,0.000263,0.000348,-0.000052,-0.000270,-0.000334,0.000353,0.000310
1,0,0.000070,-0.000236,0.000249,0.000155,-0.000142,-0.000312,-0.000089,0.000149,0.000301,...,0.000041,-0.000108,0.000068,0.000095,-0.000318,0.000315,0.000111,-0.000155,0.000010,0.000061
2,0,0.000121,-0.000287,0.000169,0.000058,-0.000101,0.000159,-0.000277,0.000111,-0.000060,...,-0.000123,0.000265,0.000006,0.000012,-0.000199,0.000148,0.000175,-0.000399,0.000139,-0.000191
3,0,-0.000215,-0.000446,-0.000029,-0.000113,0.000101,0.000136,0.000362,-0.000373,-0.000048,...,0.000026,-0.000055,0.000263,-0.000010,-0.000090,-0.000160,-0.000041,-0.000250,0.000085,-0.000165
4,0,0.000272,-0.000170,0.000101,-0.000099,0.000003,-0.000267,-0.000073,-0.000203,0.000183,...,-0.000057,-0.000156,0.000129,-0.000020,0.000163,0.000375,0.000178,-0.000314,-0.000084,-0.000133


In [4]:
y = df['0']
x = df.drop('0', axis=1) 
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [5]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.731767  0.481506   
1  XGBClassifier(base_score=None, booster=None, c...  0.705835  0.414268   
2      LGBMClassifier(max_depth=10, random_state=50)  0.686386  0.374051   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.676661  0.353457   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.606159  0.212860   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.463533   0.817778  0.596434  0.689784     0.867099     0.596434  
1  0.411669   0.731752  0.649919  0.688412     0.761750     0.649919  
2  0.372771   0.703180  0.645057  0.672866     0.727715     0.645057  
3  0.353323   0.671924  0.690438  0.681055     0.662885     0.690438  
4  0.212318   0.599092  0.641815  0.619718     0.570502     0.641815  
